- MoE에 이르는 변천사 정리
1. 기존 선형 계층으로만 이뤄진 FFN
2. Gating Network를 추가해, 불필요한 정보를 억제함으로써 학습 효율성과 다양한 표현력을 챙긴 GLU FFN
3. 전문가 집합으로 구분해 입력 데이터에 따라 활성화되는 전문가 층을 제어함으로써 자원 효율성과 실행속도, 더 큰 표현력의 장점을 갖춘 MoE

### 1. 기본 Transformer의 FFN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicFFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super(BasicFFN, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# 예시 사용
d_model = 512
d_ff = 2048
x = torch.randn(64, d_model)  # 배치 크기 64, 모델 차원 512
ffn = BasicFFN(d_model, d_ff)
output = ffn(x)
print(output.shape)  # 출력: torch.Size([64, 512])

torch.Size([64, 512])


### 2. GLU (Gated Linear Unit)를 사용한 FFN

In [2]:
class GLUFFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super(GLUFFN, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_model, d_ff)
        self.linear3 = nn.Linear(d_ff, d_model)
        self.silu = nn.SiLU()  # Sigmoid Linear Unit (SiLU, Swish)

    def forward(self, x):
        gate = self.silu(self.linear2(x))
        value = self.linear1(x)
        x = value * gate
        x = self.linear3(x)
        return x

# 예시 사용
glu_ffn = GLUFFN(d_model, d_ff)
output_glu = glu_ffn(x)
print(output_glu.shape)  # 출력: torch.Size([64, 512])


torch.Size([64, 512])


### 3. MoE (Mixture of Experts)를 사용한 FFN

In [3]:
class MoEFFN(nn.Module):
    def __init__(self, d_model, d_ff, num_experts):
        super(MoEFFN, self).__init__()
        self.experts = nn.ModuleList([nn.Linear(d_model, d_ff) for _ in range(num_experts)])
        self.gates = nn.Linear(d_model, num_experts)
        self.linear_out = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # 각 입력에 대한 게이트 값 계산
        gate_values = F.softmax(self.gates(x), dim=-1)  # [batch_size, num_experts]
        
        # 각 전문가의 출력을 구하고 게이트로 가중합 계산
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=1)  # [batch_size, num_experts, d_ff]
        weighted_sum = torch.einsum('be,bed->bd', gate_values, expert_outputs)  # [batch_size, d_ff]

        # 최종 출력 레이어 통과
        output = self.linear_out(weighted_sum)
        return output

# 예시 사용
num_experts = 4
moe_ffn = MoEFFN(d_model, d_ff, num_experts)
output_moe = moe_ffn(x)
print(output_moe.shape)  # 출력: torch.Size([64, 512])

torch.Size([64, 512])
